# Lee un año de datos a ver cómo se hace más rápido

In [1]:
### Probando con datos
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ReadParquetYearMon16MM").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/29 17:36:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Tue Nov 29 17:36:31 2022


In [3]:
%%time
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import sum, col, desc

esquema = StructType() \
      .add("Id",StringType(),True) \
      .add("fecNac",StringType(),True) \
      .add("sex",StringType(),True) \
      .add("ingresos",IntegerType(),True) \
      .add("partido",StringType(),True) \
      .add("edad",IntegerType(),True) \
      .add("mapa",StringType(),True) \
      .add("nombre",StringType(),True) \
      .add("lider",StringType(),True) \
      .add("fechaCreacion",StringType(),True)

"""
 * Bucle año/mes
 
 1.- Lee un año mes a mes: lo hace como a menos de un segundo por mes.
 2.- Le pongo 10 años
"""
total = 0

for ano in range(1931, 2005, 10):
    for mes in range(1, 2):
        if ano == 2004 and mes > 4: break
        
        if mes > 9: anomes = str(ano)+'-'+str(mes)
        else:       anomes = str(ano)+'-0'+str(mes)
        
        anomes = str(ano)[:-1]+'*'
    
        print(132*'='+'\nTratando partición yearMon='+anomes)
        inicioP = time.perf_counter()

        df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fakePeopleConPartido.parquet/")\
                       .format("parquet").load(['./datos/fakePeopleConPartido.parquet/yearMon='+anomes])

        ct = df.count()
        print(ct)
        total = total + ct
        print ('Final partición: '+time.strftime("%c"))
        final = time.perf_counter()
        print(f'Hecho en {round(final - inicioP, 4)} segundo(s)')


Tratando partición yearMon=193*


96397918
Final partición: Tue Nov 29 17:37:58 2022
Hecho en 86.8622 segundo(s)
Tratando partición yearMon=194*


96371754
Final partición: Tue Nov 29 17:38:53 2022
Hecho en 54.9161 segundo(s)
Tratando partición yearMon=195*


96384479
Final partición: Tue Nov 29 17:39:44 2022
Hecho en 51.8633 segundo(s)
Tratando partición yearMon=196*


96383619
Final partición: Tue Nov 29 17:40:38 2022
Hecho en 53.2218 segundo(s)
Tratando partición yearMon=197*


92772373
Final partición: Tue Nov 29 17:41:28 2022
Hecho en 49.9723 segundo(s)
Tratando partición yearMon=198*


89170618
Final partición: Tue Nov 29 17:42:20 2022
Hecho en 52.1967 segundo(s)
Tratando partición yearMon=199*


89153355
Final partición: Tue Nov 29 17:43:09 2022
Hecho en 49.5377 segundo(s)
Tratando partición yearMon=200*


48488296
Final partición: Tue Nov 29 17:43:33 2022
Hecho en 23.3418 segundo(s)
CPU times: user 276 ms, sys: 38.3 ms, total: 314 ms
Wall time: 7min 1s


In [4]:
print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

print('TOTAL de registros leidos '+str(total))

Final: Tue Nov 29 17:43:33 2022
Hecho en 421.9781 segundo(s)
TOTAL de registros leidos 705122412


In [5]:
spark.sparkContext.stop()
print('Sacabao')

Sacabao
